# Radial Basis Function Networks (RBFN)

In [2]:
!pip install ucimlrepo

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [32]:
iris = fetch_ucirepo(id=53)

In [33]:
X = iris.data.features.values
y = iris.data.targets.values.ravel()
y, _ = pd.factorize(y)

In [34]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [35]:
X_train, X_test, y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [36]:
def to_tensor(data,target):
  return torch.tensor(data,dtype=torch.float32), torch.tensor(target,dtype=torch.long)

In [37]:
X_train,y_train = to_tensor(X_train,y_train)
X_test,y_test = to_tensor(X_test,y_test)

In [38]:
def rbf_kernel(X,centers,beta):
  return torch.exp(-beta*torch.cdist(X,centers)**2)

class RBFN(nn.Module):
  def __init__(self, input_dim, num_centers,output_dim):
    super(RBFN, self).__init__()
    self.centers = nn.Parameter(torch.randn(num_centers,input_dim))
    #start rbf centers randomly
    self.beta = nn.Parameter(torch.ones(1)*2.0)
    #beta controls rbf's size
    self.linear = nn.Linear(num_centers,output_dim) #output to fully connected layers

  def forward(self, X):
    phi = rbf_kernel(X,self.centers,self.beta) #calculate rbf kernel function
    return self.linear(phi)

In [39]:
num_centers = 10
model = RBFN(input_dim=4,num_centers=num_centers,output_dim=3)

In [40]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

In [41]:
num_epochs = 100
for epoch in range(num_epochs):
  optimizer.zero_grad()
  outputs = model(X_train)
  loss = criterion(outputs, y_train)
  loss.backward()
  optimizer.step()

  if (epoch + 1)%10 == 0:
    print(f"Epoch {epoch+1}/{num_epochs}, loss:{loss.item():.4f}")

Epoch 10/100, loss:1.0934
Epoch 20/100, loss:1.0816
Epoch 30/100, loss:1.0524
Epoch 40/100, loss:0.9858
Epoch 50/100, loss:0.8682
Epoch 60/100, loss:0.7286
Epoch 70/100, loss:0.5967
Epoch 80/100, loss:0.4741
Epoch 90/100, loss:0.3750
Epoch 100/100, loss:0.3013


In [42]:
with torch.no_grad():
  y_pred = model(X_test)
  accuracy = (torch.argmax(y_pred,axis=1)==y_test).float().mean().item()
  print(f"Accuracy {accuracy}")

Accuracy 0.9555555582046509
